# Semantinis branduolys su OpenBnB MCP serverio integracija

Šis užrašų knygelės pavyzdys demonstruoja, kaip naudoti Semantinį branduolį su tikruoju OpenBnB MCP serveriu, kad būtų ieškoma tikrų Airbnb apgyvendinimo vietų naudojant MCPStdioPlugin. LLM prieigai jis naudoja Microsoft Foundry. Norėdami nustatyti savo aplinkos kintamuosius, galite vadovautis [Setup Lesson ](/00-course-setup/README.md)


## Importuoti reikalingas paketas


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP papildinio prisijungimo kūrimas

Prisijungsime prie [OpenBnB MCP serverio](https://github.com/openbnb-org/mcp-server-airbnb) naudodami MCPStdioPlugin. Šis serveris teikia Airbnb paieškos funkcionalumą per @openbnb/mcp-server-airbnb paketą.


## Kliento kūrimas

Šiame pavyzdyje naudosime Microsoft Foundry mūsų LLM prieigai. Įsitikinkite, kad jūsų aplinkos kintamieji nustatyti teisingai.


## Aplinkos konfigūravimas

Konfigūruokite Azure OpenAI nustatymus. Įsitikinkite, kad nustatėte šiuos aplinkos kintamuosius:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP integracijos supratimas

Šis užrašų knygutė jungiasi prie **tikrojo OpenBnB MCP serverio**, kuris teikia faktinę Airbnb paieškos funkcionalumą.

### Kaip tai veikia:

1. **MCPStdioPlugin**: Naudoja standartinę įvesties/išvesties komunikaciją su MCP serveriu  
2. **Tikras NPM paketas**: Atsisiunčia ir paleidžia `@openbnb/mcp-server-airbnb` per npx  
3. **Gyvi duomenys**: Grąžina faktinius Airbnb nekilnojamojo turto duomenis iš jų API  
4. **Funkcijų aptikimas**: Agentas automatiškai aptinka galimas funkcijas iš MCP serverio

### Galimos funkcijos:

OpenBnB MCP serveris paprastai teikia:  
- **search_listings** – Ieškoti Airbnb objektų pagal vietovę ir kriterijus  
- **get_listing_details** – Gauti išsamią informaciją apie konkrečius objektus  
- **check_availability** – Patikrinti prieinamumą konkrečioms datoms  
- **get_reviews** – Gauti atsiliepimus apie objektus  
- **get_host_info** – Gauti informacijos apie savininkus

### Reikalavimai:

- **Node.js** įdiegtas jūsų sistemoje  
- **Interneto ryšys** MCP serverio paketo atsisiuntimui  
- **NPX** prieinamas (yra Node.js dalis)

### Ryšio testavimas:

MCP serverį galite išbandyti rankiniu būdu paleisdami:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
Tai atsisiųs ir paleis OpenBnB MCP serverį, prie kurio tada prisijungs Semantic Kernel, kad gautų tikrus Airbnb duomenis.


## Agentas paleidimas su OpenBnB MCP serveriu

Dabar paleisime AI Agentą, kuris jungiasi prie OpenBnB MCP serverio ieškoti tikrų Airbnb apgyvendinimo vietų Stokholme 2 suaugusiems ir 1 vaikui. Nesivaržykite keisti `user_inputs` sąrašo, norėdami pakeisti paieškos kriterijus.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Santrauka
Sveikiname! Jūs sėkmingai sukūrėte AI agentą, kuris integruojasi su realaus pasaulio apgyvendinimo paieška naudojant Model Context Protocol (MCP):

## Naudotos technologijos:
- Semantic Kernel – Inteli­gen­tiški­ų agentų kūrimui su Azure OpenAI
- Microsoft Foundry – LLM galimybėms ir pokalbių užbaigimui
- MCP (Model Context Protocol) – Standartizuotai įrankių integracijai
- OpenBnB MCP serveris – Tikrų Airbnb paieškos funkcionalumui
- Node.js/NPX – Išorinio MCP serverio paleidimui

## Ko išmokote:
- MCP integracija: Semantic Kernel agentų prijungimas prie išorinių MCP serverių
- Realaus laiko duomenų prieiga: Tikrų Airbnb nekilnojamojo turto paieška per gyvus API
- Protokolo komunikacija: Naudojant stdio ryšį tarp agento ir MCP serverio
- Funkcijų aptikimas: Automatinis galimų funkcijų aptikimas iš MCP serverių
- Srautinės atsakymų perdavimas: Funkcijų kvietimų fiksavimas ir registravimas realiu laiku
- HTML atvaizdavimas: Agentų atsakymų formatavimas su stilizuotomis lentelėmis ir interaktyviais rodiniais

## Kiti žingsniai:
- Integruoti papildomus MCP serverius (oras, skrydžiai, restoranai)
- Kurti daugiaagentę sistemą, derinant MCP ir A2A protokolus
- Kurti individualius MCP serverius savo duomenų šaltiniams
- Įgyvendinti nuolatinės pokalbių atminties palaikymą per sesijas
- Diegti agentą Azure Functions su MCP serverio orkestravimu
- Pridėti naudotojų autentifikaciją ir užsakymų galimybes


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatizuotas vertimas gali turėti klaidų arba netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Dėl svarbios informacijos rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neatsakome už jokius nesusipratimus ar neteisingus aiškinimus, kylančius naudojant šį vertimą.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
